# 实验1_神经网络原理与实现


## 1. iris分类识别

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

In [3]:
# 加载并预处理Iris数据集
iris = load_iris()
X, y = iris.data, iris.target

# 标准化数据
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [9]:
# 划分训练集和测试集 (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 转换为PyTorch张量
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

In [10]:
# 定义神经网络模型
class IrisNet(nn.Module):
    def __init__(self):
        super(IrisNet, self).__init__()
        # 输入层: 4个特征 -> 隐藏层1: 10个神经元
        self.fc1 = nn.Linear(4, 10)
        # 隐藏层1 -> 隐藏层2: 5个神经元
        self.fc2 = nn.Linear(10, 5)
        # 隐藏层2 -> 输出层: 3个类别
        self.fc3 = nn.Linear(5, 3)

        # 激活函数 (ReLU)
        self.relu = nn.ReLU()

    def forward(self, x):
        # 输入 -> 隐藏层1
        x = self.relu(self.fc1(x))
        # 隐藏层1 -> 隐藏层2
        x = self.relu(self.fc2(x))
        # 隐藏层2 -> 输出层
        x = self.fc3(x)
        return x

In [11]:
#初始化模型、优化器和损失函数
model = IrisNet()
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()  # 自动处理Softmax和交叉熵

In [12]:
# 训练模型
num_epochs = 200
batch_size = 16

for epoch in range(num_epochs):
    # 随机打乱数据
    indices = torch.randperm(X_train.size(0))
    X_train = X_train[indices]
    y_train = y_train[indices]

    # 批量训练
    for i in range(0, X_train.size(0), batch_size):
        batch_x = X_train[i:i+batch_size]
        batch_y = y_train[i:i+batch_size]

        # 前向传播
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)

        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # 每10轮打印一次损失
    if (epoch + 1) % 10 == 0:
        with torch.no_grad():
            train_loss = criterion(model(X_train), y_train).item()
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss:.4f}')

Epoch [10/200], Loss: 0.1074
Epoch [20/200], Loss: 0.0640
Epoch [30/200], Loss: 0.0492
Epoch [40/200], Loss: 0.0497
Epoch [50/200], Loss: 0.0438
Epoch [60/200], Loss: 0.0448
Epoch [70/200], Loss: 0.0431
Epoch [80/200], Loss: 0.0430
Epoch [90/200], Loss: 0.0433
Epoch [100/200], Loss: 0.0430
Epoch [110/200], Loss: 0.0414
Epoch [120/200], Loss: 0.0412
Epoch [130/200], Loss: 0.0472
Epoch [140/200], Loss: 0.0412
Epoch [150/200], Loss: 0.0601
Epoch [160/200], Loss: 0.0412
Epoch [170/200], Loss: 0.0419
Epoch [180/200], Loss: 0.0438
Epoch [190/200], Loss: 0.0410
Epoch [200/200], Loss: 0.0411


In [13]:
# 测试模型
with torch.no_grad():
    outputs = model(X_test)
    _, predicted = torch.max(outputs.data, 1)
    accuracy = (predicted == y_test).sum().item() / y_test.size(0)
    print(f'Test Accuracy: {accuracy:.4f}')

Test Accuracy: 1.0000


In [14]:
# 验证预测示例
sample = X_test[0:1]
with torch.no_grad():
    output = model(sample)
    _, pred = torch.max(output, 1)
    print(f'\nSample prediction (actual: {y_test[0].item()}): {pred.item()}')


Sample prediction (actual: 1): 1


In [16]:
# 保存模型
torch.save(model.state_dict(), './models/iris_model.pth')
print("模型已保存为 'iris_model.pth'")

模型已保存为 'iris_model.pth'


## 2. 手写数字识别
- 调整参数，比较不同模型分类性能

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

# 设置设备（GPU 或 CPU）
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用设备: {device}")

# 数据预处理和加载
transform = transforms.Compose([
    transforms.ToTensor(),  # 转为 [0,1] 的张量
    transforms.Normalize((0.1307,), (0.3081,))  # MNIST 全局均值和标准差
])


使用设备: cpu


In [2]:

# 下载并加载 MNIST 数据集
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# 创建数据加载器
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

100.0%
100.0%
100.0%
100.0%


In [3]:
# 定义神经网络模型
class MNISTNet(nn.Module):
    def __init__(self):
        super(MNISTNet, self).__init__()
        self.fc1 = nn.Linear(28*28, 512)   # 输入层 -> 隐藏层
        self.fc2 = nn.Linear(512, 10)      # 隐藏层 -> 输出层
        self.relu = nn.ReLU()

    def forward(self, x):
        x = x.view(x.size(0), -1)  # 展平图像 (64, 1, 28, 28) -> (64, 784)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# 初始化模型、损失函数、优化器
model = MNISTNet().to(device)
criterion = nn.CrossEntropyLoss()



In [4]:
# 训练函数
def train(epoch):
    model.train()
    total_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f'Epoch [{epoch+1}], Average Loss: {avg_loss:.4f}')
    return avg_loss

# 测试函数
def test():
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            _, predicted = torch.max(output, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

    accuracy = 100 * correct / total
    print(f'Test Accuracy: {accuracy:.2f}%\n')
    return accuracy

In [5]:
lr = 0.001
optimizer = optim.Adam(model.parameters(), lr=lr)

print(f"模型结构:\n{model}")
print(f"学习率: {lr}")



# 开始训练
num_epochs = 5
train_losses = []
test_accuracies = []

for epoch in range(num_epochs):
    loss = train(epoch)
    acc = test()
    train_losses.append(loss)
    test_accuracies.append(acc)

模型结构:
MNISTNet(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=10, bias=True)
  (relu): ReLU()
)
学习率: 0.001
Epoch [1], Average Loss: 0.2034
Test Accuracy: 96.95%

Epoch [2], Average Loss: 0.0832
Test Accuracy: 97.81%

Epoch [3], Average Loss: 0.0579
Test Accuracy: 98.08%

Epoch [4], Average Loss: 0.0397
Test Accuracy: 97.80%

Epoch [5], Average Loss: 0.0325
Test Accuracy: 97.90%



In [6]:
lr = 0.01  # 调整学习率
optimizer = optim.Adam(model.parameters(), lr=lr)

print(f"模型结构:\n{model}")
print(f"学习率: {lr}")



# 开始训练
num_epochs = 5
train_losses = []
test_accuracies = []

for epoch in range(num_epochs):
    loss = train(epoch)
    acc = test()
    train_losses.append(loss)
    test_accuracies.append(acc)

模型结构:
MNISTNet(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=10, bias=True)
  (relu): ReLU()
)
学习率: 0.01
Epoch [1], Average Loss: 0.2861
Test Accuracy: 94.75%

Epoch [2], Average Loss: 0.2173
Test Accuracy: 93.92%

Epoch [3], Average Loss: 0.1944
Test Accuracy: 94.98%

Epoch [4], Average Loss: 0.1797
Test Accuracy: 94.53%

Epoch [5], Average Loss: 0.1845
Test Accuracy: 94.44%

